In [1]:
from pathlib import Path
import joblib
from typing import List

import pandas as pd
from sklearn.pipeline import Pipeline

from pipeline import (
    DropColumns,
    DropHighNAPercentage,
    NormalizeCurrency,
    OrdinalColumnMapper,
    DataframeOneHotEncoder,
    NanInputer,
    InfoDisplayer,
    MultiDataFramePipeline,
    MultiDataFrameMerger,
)
from utils import (
    show,
    show_null_percentages,
)

pd.set_option("display.max_columns", None)

In [2]:
ROOT_DIR = Path.cwd().parent
DATA_DIR = ROOT_DIR / "data"

DATA_PATH = DATA_DIR / 'indices_institucional_2005_2022.xlsx'
DATA_PATH.exists()

True

In [3]:
df = joblib.load(DATA_DIR / "base_indices.pkl")
df = df.reset_index()
df = df[df["Promedio Puntaje (promedio matemáticas y lenguaje)"].notna()].reset_index(
    drop=True
)
df = (
    df[df["Pregrado/Posgrado"] == "Pregrado"]
    .drop(columns=["Pregrado/Posgrado"])
    .reset_index(drop=True)
    # .rename(columns={"Año": "Fecha"})
)
df["Año"] = df["Año"].dt.year

In [4]:
inmuebles, labs, docentes = joblib.load(DATA_DIR / "extra_data.pkl")


def prepare_for_merge(df_to_merge: pd.DataFrame, year_col: str) -> pd.DataFrame:
    df_to_merge["Año"] = df_to_merge[year_col].astype(int)
    df_to_merge = df_to_merge.drop(columns=[year_col])
    df_to_merge = df_to_merge.rename(columns={"idSede": "Cód. Sede"})
    df_to_merge["Cód. Sede"] = df_to_merge["Cód. Sede"].astype(int)
    df_to_merge["Cód. Institución"] = df_to_merge["Cód. Institución"].astype(int)
    df_to_merge = df_to_merge.drop(columns=["Tipo Institución", "Cód. Institución"])

    return df_to_merge


inmuebles = prepare_for_merge(inmuebles, year_col="Año Información")
labs = prepare_for_merge(labs, year_col="Año Proceso")
docentes = prepare_for_merge(docentes, year_col="Año Proceso")

### Main Data Pipeline

In [5]:
# DROP COLUMNS
COLUMNS_TO_DROP = [
    # drop por no ser relevantes
    "Nombre de la Sede",
    "Orden Geográfico de la Región (Norte aSur)",
    "Mención o Especialidad",
    "idgenerocarrera",
    "Códgo SIES",
    "Máximo Puntaje (promedio matemáticas y lenguaje)",
    "Máximo Puntaje NEM",
    "Máximo Puntaje Ranking",
    "Mínimo Puntaje (promedio matemáticas y lenguaje)",
    "Mínimo Puntaje NEM",
    "Mínimo Puntaje Ranking",
    # drop por decision de mining
    "Grado Académico",
    "Cód. Institución",
    "Nombre Region",
    "Carrera Genérica",
    "Cód. Carrera",
    "Nombre Programa",
    "Nombre del Campus",
    "Título",
]

# DROP HIGH NA COLUMNS
EXCLUDE_COLUMNS_OF_DROPHIGHNA = ["Nombre del Campus"]

# NORMALIZE CURRENCY
CURRENCY_COLUMNS = [
    "Valor de matrícula",
    "Valor de arancel",
    "Valor del Título",
]

# ORDINAL ENCODER
m_class1 = {
    "(a) Universidades CRUCH": 0,
    "(b) Universidades Privadas": 1,
    "(c) Institutos Profesionales": 2,
    "(d) Centros de Formación Técnica": 3,
    "(e) Centros de Formación Técnica Estatales": 3,
    "(f) F.F.A.A.": 4,
}

m_class2 = {
    "(a) Universidades Estatales CRUCH": 0,
    "(b) Universidades Privadas CRUCH": 1,
    "(c) Univ. Privadas Adscritas SUA": 2,
    "(d) Universidades Privadas": 3,
    "(e) Institutos Profesionales": 4,
    "(f) Centros de Formación Técnica": 5,
    "(g) Centros de Formación Técnica statales": 5,
    "(h) F.F.A.A.": 6,
}

m_class3 = {"(a) Acreditada": 0, "(b) No Acreditada": 1}

m_class4 = {
    "(a) Autónoma": 0,
    "(b) Licenciamiento": 1,
    "(c) Examinación": 2,
    "(d) Supervisión": 3,
    "(e) F.F.A.A.": 4,
    "(e) Cerrada": 5,
}

m_class5 = {"(a) Adscritas a Gratuidad": 0, "(b) No Adscritas/No Aplica": 1}

m_class6 = {
    "(a) Subsistema Universitario": 0,
    "(b) Subsistema Técnico Profesional": 1,
    "(c) No adscrito": 2,
    "(d) F.F.A.A.": 3,
}

m_inst = {
    "Univ.": 0,
    "I.P.": 1,
    "C.F.T.": 2,
    "F.F.A.A.": 3,
}

m_tipo_programa = {"Programa Regular": 0, "Programa Especial": 1}

m_tipo_carrera = {
    "Profesional con Licenciatura": 0,
    "Técnico Nivel Superior": 3,
    "Profesional": 1,
    "Licenciatura": 2,
    "Bachillerato": 4,
    "Plan Común o Ciclo Básico": 4,
}

m_tipo_ingreso = {"Ingreso Directo": 0, "No es Ingreso Directo": 1}

COLUMNS_TO_MAP = [f"Clasificación{i}" for i in range(1, 7)]
COLUMNS_TO_MAP.extend(
    [
        "Tipo Institución",
        "Tipo Programa",
        "Tipo Carrera",
        "IngresoDirecto",
    ]
)
MAPPINGS = [
    m_class1,
    m_class2,
    m_class3,
    m_class4,
    m_class5,
    m_class6,
    m_inst,
    m_tipo_programa,
    m_tipo_carrera,
    m_tipo_ingreso,
]

# ONE HOT ENCODER
COLUMNS_TO_ONE_HOT = [
    "Nombre Institución",
    "Comuna donde se imparte la carrera o programa",
    "Area Conocimiento",
    "Horario",
]

In [6]:
data_pipeline = Pipeline(
    steps=[
        # ("info_displayer1", InfoDisplayer(name="Original Data")),
        (
            "drop_columns",
            DropColumns(
                columns=COLUMNS_TO_DROP,
            ),
        ),
        # ("info_displayer2", InfoDisplayer(name="After Drop Columns")),
        (
            "drop_high_na",
            DropHighNAPercentage(
                na_threshold=0.24, exclude=EXCLUDE_COLUMNS_OF_DROPHIGHNA
            ),
        ),
        # ("info_displayer3", InfoDisplayer(name="After Drop High NA")),
        (
            "preprocess_tipo_moneda",
            NormalizeCurrency(columns=CURRENCY_COLUMNS),
        ),
        # ("info_displayer4", InfoDisplayer(name="After Normalize Currency")),
        (
            "ordinal_encoder",
            OrdinalColumnMapper(columns=COLUMNS_TO_MAP, mappings=MAPPINGS),
        ),
        # ("info_displayer5", InfoDisplayer(name="After Ordinal Encoder")),
        (
            "one_hot_encoder",
            DataframeOneHotEncoder(
                columns=COLUMNS_TO_ONE_HOT,
                min_frequency=20,
                # max_categories=30,
            ),
        ),
        # ("info_displayer6", InfoDisplayer(name="After One Hot Encoder")),
        (
            "nan_inputer",
            NanInputer(n_neighbors=5, columns="auto"),
        ),
        # ("info_displayer7", InfoDisplayer(name="After Nan Inputer")),
    ],
    # verbose=True,
)

### Data to merge pipelines

In [7]:
inmuebles_pipe = Pipeline(
    steps=[
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

labs_pipe = Pipeline(
    steps=[
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

docentes_pipe = Pipeline(
    steps=[
        (
            "drop_columns",
            DropColumns(
                columns=["Nombre Institución", "Sede", "idInstitucion"], errors="ignore"
            ),
        ),
        ("drop_high_na", DropHighNAPercentage(na_threshold=0.3, exclude=[])),
        ("inputer", NanInputer(n_neighbors=5)),
    ]
)

In [8]:
# for pipe in [data_pipeline, inmuebles_pipe, labs_pipe, docentes_pipe]:
#     display(pipe)

In [9]:
MERGE_COLS = [
    "Año",
    "Cód. Sede",
]

multi_pipe = MultiDataFramePipeline(
    transformers=[
        ("data_processor", data_pipeline),
        ("inmuebles_processor", inmuebles_pipe),
        ("labs_processor", labs_pipe),
        ("docentes_processor", docentes_pipe),
    ]
)

pipe_merger = MultiDataFrameMerger(columns=MERGE_COLS)

data_processor = Pipeline(
    [
        ("multi_pipe", multi_pipe),
        ("pipe_merger", pipe_merger),
    ]
)
    

In [10]:
processed_data = data_processor.fit_transform(
    [
        df,
        inmuebles,
        labs,
        docentes,
    ]
)

Fitting Pipeline...
Fitting Pipeline...
Fitting Pipeline...
Fitting Pipeline...
Returning 4 DataFrames.
Fitting MultiDataFrameMerger...


In [12]:
show(processed_data)

(30607, 187)


,Año,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Tipo Programa,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Duración (en semestres),Cód. Sede,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total,Nombre Institución_C.F.T. DIEGO PORTALES,Nombre Institución_C.F.T. JOHN F. KENNEDY,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. SIMÓN BOLIVAR,Nombre Institución_ESCUELA DE AVIACIÓN,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_PONTIFICIA U. CATÓLICA DE VALPARAÍSO,Nombre Institución_U. ACADEMIA DE HUMANISMO CRISTIANO,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ADVENTISTA DE CHILE,Nombre Institución_U. ALBERTO HURTADO,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BERNARDO O`HIGGINS,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA CARDENAL RAÚL SILVA HENRÍQUEZ,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DE TEMUCO,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. CHILENO-BRITÁNICA DE CULTURA,Nombre Institución_U. DE ANTOFAGASTA,Nombre Institución_U. DE ARTE Y CIENCIAS SOCIALES ARCIS,Nombre Institución_U. DE ATACAMA,Nombre Institución_U. DE AYSEN,Nombre Institución_U. DE CHILE,Nombre Institución_U. DE CONCEPCIÓN,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS ANDES,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE O`HIGGINS,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE SANTIAGO DE CHILE,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. DIEGO PORTALES,Nombre Institución_U. FINIS TERRAE,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. IBEROAMERICANA DE CIENCIAS Y TECNOLOGÍA,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MARÍTIMA DE CHILE,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. SEK,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_U. UCINF,Nombre Institución_infrequent_sklearn,Comuna donde se imparte la carrera o programa_Antofagasta,Comuna donde se imparte la carrera o programa_Arica,Comuna donde se imparte la carrera o programa_Calama,Comuna donde se imparte la carrera o programa_Chillán,Comuna donde se imparte la carrera o programa_Concepción,Comuna donde se imparte la carrera o programa_Copiapó,Comuna donde se imparte la carrera o programa_Coquimbo,Comuna donde se imparte la carrera o programa_Coyhaique,Comuna donde se imparte la carrera o programa_Curicó,Comuna donde se imparte la carrera o programa_El Bosque,Comuna donde se imparte la carre

In [14]:
processed_data.describe()

,Año,Tipo Institución,Clasificación1,Clasificación2,Clasificación3,Clasificación4,Clasificación5,Clasificación6,Tipo Programa,Tipo Carrera,IngresoDirecto,Año Inicio Actividades,Duración (en semestres),Cód. Sede,Promedio Puntaje (promedio matemáticas y lenguaje),Puntaje de corte (primer seleccionado),Puntaje de corte (promedio de la carrera),Puntaje de corte (último seleccionado),Nº Alumnos Ingreso Via PSU o PDT,Valor de matrícula,Valor de arancel,Valor del Título,Vacantes,Matrícula primer año hombres,Matrícula primer año mujeres,Matrícula Primer Año,Matrícula total hombres,Matrícula total mujeres,Matrícula Total,Nombre Institución_C.F.T. DIEGO PORTALES,Nombre Institución_C.F.T. JOHN F. KENNEDY,Nombre Institución_C.F.T. SANTO TOMÁS,Nombre Institución_C.F.T. SIMÓN BOLIVAR,Nombre Institución_ESCUELA DE AVIACIÓN,Nombre Institución_I.P. AIEP,Nombre Institución_I.P. DEL VALLE CENTRAL,Nombre Institución_I.P. DIEGO PORTALES,Nombre Institución_I.P. ESCUELA MODERNA DE MÚSICA,Nombre Institución_I.P. GUILLERMO SUBERCASEAUX,Nombre Institución_I.P. LOS LEONES,Nombre Institución_I.P. SANTO TOMÁS,Nombre Institución_PONTIFICIA U. CATÓLICA DE CHILE,Nombre Institución_PONTIFICIA U. CATÓLICA DE VALPARAÍSO,Nombre Institución_U. ACADEMIA DE HUMANISMO CRISTIANO,Nombre Institución_U. ADOLFO IBÁÑEZ,Nombre Institución_U. ADVENTISTA DE CHILE,Nombre Institución_U. ALBERTO HURTADO,Nombre Institución_U. ANDRÉS BELLO,Nombre Institución_U. ARTURO PRAT,Nombre Institución_U. AUSTRAL DE CHILE,Nombre Institución_U. AUTÓNOMA DE CHILE,Nombre Institución_U. BERNARDO O`HIGGINS,Nombre Institución_U. BOLIVARIANA,Nombre Institución_U. CATÓLICA CARDENAL RAÚL SILVA HENRÍQUEZ,Nombre Institución_U. CATÓLICA DE LA SANTÍSIMA CONCEPCIÓN,Nombre Institución_U. CATÓLICA DE TEMUCO,Nombre Institución_U. CATÓLICA DEL MAULE,Nombre Institución_U. CATÓLICA DEL NORTE,Nombre Institución_U. CENTRAL DE CHILE,Nombre Institución_U. CHILENO-BRITÁNICA DE CULTURA,Nombre Institución_U. DE ANTOFAGASTA,Nombre Institución_U. DE ARTE Y CIENCIAS SOCIALES ARCIS,Nombre Institución_U. DE ATACAMA,Nombre Institución_U. DE AYSEN,Nombre Institución_U. DE CHILE,Nombre Institución_U. DE CONCEPCIÓN,Nombre Institución_U. DE LA FRONTERA,Nombre Institución_U. DE LA SERENA,Nombre Institución_U. DE LAS AMÉRICAS,Nombre Institución_U. DE LOS ANDES,Nombre Institución_U. DE LOS LAGOS,Nombre Institución_U. DE MAGALLANES,Nombre Institución_U. DE O`HIGGINS,Nombre Institución_U. DE PLAYA ANCHA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. DE SANTIAGO DE CHILE,Nombre Institución_U. DE TALCA,Nombre Institución_U. DE TARAPACÁ,Nombre Institución_U. DE VALPARAÍSO,Nombre Institución_U. DE VIÑA DEL MAR,Nombre Institución_U. DEL ALBA,Nombre Institución_U. DEL BÍO-BÍO,Nombre Institución_U. DEL DESARROLLO,Nombre Institución_U. DEL MAR,Nombre Institución_U. DEL PACÍFICO,Nombre Institución_U. DIEGO PORTALES,Nombre Institución_U. FINIS TERRAE,Nombre Institución_U. GABRIELA MISTRAL,Nombre Institución_U. IBEROAMERICANA DE CIENCIAS Y TECNOLOGÍA,Nombre Institución_U. LA REPÚBLICA,Nombre Institución_U. MARÍTIMA DE CHILE,Nombre Institución_U. MAYOR,Nombre Institución_U. METROPOLITANA DE CIENCIAS DE LA EDUCACIÓN,Nombre Institución_U. SAN SEBASTIÁN,Nombre Institución_U. SANTO TOMÁS,Nombre Institución_U. SEK,Nombre Institución_U. TECNOLÓGICA METROPOLITANA,Nombre Institución_U. TÉCNICA FEDERICO SANTA MARÍA,Nombre Institución_U. UCINF,Nombre Institución_infrequent_sklearn,Comuna donde se imparte la carrera o programa_Antofagasta,Comuna donde se imparte la carrera o programa_Arica,Comuna donde se imparte la carrera o programa_Calama,Comuna donde se imparte la carrera o programa_Chillán,Comuna donde se imparte la carrera o programa_Concepción,Comuna donde se imparte la carrera o programa_Copiapó,Comuna donde se imparte la carrera o programa_Coquimbo,Comuna donde se imparte la carrera o programa_Coyhaique,Comuna donde se imparte la carrera o programa_Curicó,Comuna donde se imparte la carrera o programa_El Bosque,Comuna donde se imparte la carre

In [11]:
multi_pipe.transformers

[Pipeline(steps=[('info_displayer1', InfoDisplayer(name='Original Data')),
                 ('drop_columns',
                  DropColumns(columns=['Nombre de la Sede', 'Orden Geográfico de la Región (Norte aSur)', 'Mención o Especialidad', 'idgenerocarrera', 'Códgo SIES', 'Máximo Puntaje (promedio matemáticas y lenguaje)', 'Máximo Puntaje NEM', 'Máximo Puntaje Ranking', 'Mínimo Puntaje (promedio matemáticas y lenguaje)'...
                  InfoDisplayer(name='After Ordinal Encoder')),
                 ('one_hot_encoder',
                  DataframeOneHotEncoder(columns=['Nombre Institución',
                                                  'Comuna donde se imparte la '
                                                  'carrera o programa',
                                                  'Area Conocimiento',
                                                  'Horario'],
                                         min_frequency=20)),
                 ('info_displayer6',
                